In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [31]:
def get_soup(url):
    r  = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, 'html.parser')
    return soup


In [67]:
def find_french_page(soup):
    return soup.body.find_all("li", class_="interlanguage-link interwiki-fr")[0].a.get("href")

In [96]:
def get_latlong(coordinates):
    lat = float(coordinates[0][:-1]) + float(coordinates[1][:-1])/60 + float(coordinates[2][:-1])/3600
    long = float(coordinates[4][:-1]) + float(coordinates[5][:-1])/60 + float(coordinates[6][:-1])/3600
    return lat, long

In [108]:
soup = get_soup("https://en.wikipedia.org/wiki/List_of_museums_in_Paris")
df = pd.DataFrame(columns=['Museum name', 'Latitude', 'Longitude'])

In [109]:

for museum in soup.body.table.find_all('td')[::5]:
    try:
        museum_name = museum.a.get_text()
        museum_webpage = "https://en.wikipedia.org"+museum.a.get('href')
        if "redlink" not in museum_webpage:
            soup_museum = get_soup(museum_webpage)
            french_webpage = find_french_page(soup_museum)
            print(museum_name, french_webpage)    
            soup_french = get_soup(french_webpage)
            coordinates = soup_french.body.span.a.get_text().split()
            if len(coordinates) == 8:
                lat, long = get_latlong(coordinates)
                df = df.append({'Museum name': museum_name, 'Latitude':lat, 'Longitude':long}, ignore_index=True)
    except:
        pass
    
   

Arab World Institute https://fr.wikipedia.org/wiki/Institut_du_monde_arabe
Archaeological Crypt of the Paris Notre-Dame https://fr.wikipedia.org/wiki/Cath%C3%A9drale_Notre-Dame_de_Paris
Art Ludique https://fr.wikipedia.org/wiki/Art_ludique_-_Le_Mus%C3%A9e
Bibliothèque-Musée de l'Opéra National de Paris https://fr.wikipedia.org/wiki/Biblioth%C3%A8que-mus%C3%A9e_de_l%27Op%C3%A9ra
Bibliothèque Polonaise de Paris https://fr.wikipedia.org/wiki/Biblioth%C3%A8que_polonaise_de_Paris
Cabinet des Médailles https://fr.wikipedia.org/wiki/D%C3%A9partement_des_Monnaies,_m%C3%A9dailles_et_antiques_de_la_Biblioth%C3%A8que_nationale_de_France
Catacombes de Paris https://fr.wikipedia.org/wiki/Catacombes_de_Paris
Cité de l'Architecture et du Patrimoine https://fr.wikipedia.org/wiki/Cit%C3%A9_de_l%27architecture_et_du_patrimoine
Cité des Sciences et de l'Industrie https://fr.wikipedia.org/wiki/Cit%C3%A9_des_sciences_et_de_l%27industrie
Cité nationale de l'histoire de l'immigration https://fr.wikipedia.org

Musée national Gustave Moreau https://fr.wikipedia.org/wiki/Mus%C3%A9e_Gustave-Moreau
Musée national Jean-Jacques Henner https://fr.wikipedia.org/wiki/Mus%C3%A9e_national_Jean-Jacques-Henner
Musée Nissim de Camondo https://fr.wikipedia.org/wiki/Mus%C3%A9e_Nissim-de-Camondo
Musée Pasteur https://fr.wikipedia.org/wiki/Mus%C3%A9e_Pasteur
Musée Picasso https://fr.wikipedia.org/wiki/Mus%C3%A9e_Picasso_(Paris)
Musée Rodin https://fr.wikipedia.org/wiki/Mus%C3%A9e_Rodin
Musée Valentin Haüy https://fr.wikipedia.org/wiki/Mus%C3%A9e_Valentin-Ha%C3%BCy
Musée Zadkine https://fr.wikipedia.org/wiki/Mus%C3%A9e_Zadkine
Muséum national d'Histoire naturelle https://fr.wikipedia.org/wiki/Mus%C3%A9um_national_d%27histoire_naturelle
Palais de la Découverte https://fr.wikipedia.org/wiki/Palais_de_la_d%C3%A9couverte
Palais de Tokyo https://fr.wikipedia.org/wiki/Palais_de_Tokyo
Palais Galliera https://fr.wikipedia.org/wiki/Palais_Galliera,_mus%C3%A9e_de_la_Mode_de_la_ville_de_Paris
Pavillon de l'Arsenal https:

In [117]:
df.head(10)

,Museum name,Latitude,Longitude
0,Arab World Institute,48.849167,2.356944
1,Archaeological Crypt of the Paris Notre-Dame,48.853056,2.349722
2,Bibliothèque-Musée de l'Opéra National de Paris,48.871944,2.331111
3,Bibliothèque Polonaise de Paris,48.851389,2.355833
4,Cabinet des Médailles,48.867778,2.338333
5,Catacombes de Paris,48.833889,2.332222
6,Cité de l'Architecture et du Patrimoine,48.863056,2.289444
7,Cité des Sciences et de l'Industrie,48.895556,2.388056
8,Cité nationale de l'histoire de l'immigration,48.835278,2.409444
9,Espace Dalí,48.886667,2.339722


In [112]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [113]:
lat_paris, long_paris = df['Latitude'].mean(), df['Longitude'].mean()
print('Paris is at the coordinates (%s, %s).' % (lat_paris, long_paris))

Paris is at the coordinates (48.859524627720525, 2.330758877434135).


In [116]:
map_paris = folium.Map(location=[lat_paris, long_paris], zoom_start=12)

# add markers to map
for lat, lng, museum in zip(df['Latitude'], df['Longitude'], df['Museum name']):
    label = '{}'.format(museum)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

